# Effective Threshold Estimation

## Imports

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Estimators import compute_effective_threshold

#### Helper Definition

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

top = mpl.colormaps['Oranges'].resampled(64)
bottom = mpl.colormaps['Blues_r'].resampled(64)
one_centered_colors = np.vstack((bottom(np.linspace(0.0, 0.75, 64)),
                       top(np.linspace(0.25, 1.0, 64))))
one_centered_cmp = ListedColormap(one_centered_colors, name='OrangeBlue')

## Plot 1: Full Bloch sphere view

This plot provides an overview of the infidelities across the Bloch sphere.

### Scan

First, perform a scan over different error angles, applying $U(\theta, \phi)$, attempting QEC (maximum rounds: 1), and computing fidelities at each step.

In [ ]:
interior_points, fidelities, effective_threshold = compute_effective_threshold(
    **steane_code,
    hardware_model=None,
    min_theta=0, max_theta=np.pi/2, n_theta=16,
    min_phi=0, max_phi=np.pi, n_phi=8,
    shots=1E5
)

In [ ]:
fidelities

In [ ]:
effective_threshold

### Plot

Get a list of angles and infidelities for a single specified `initial_state`. We choose between plotting all data points and only plotting those that are interior points (i.e. they do not exceed the effective threshold). We can then plot the infidelities against a Bloch sphere.

In [ ]:
initial_state = "0"
plot_all = True

if plot_all:
    angles_list = list(fidelities[initial_state].keys())
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    fidelities_list = np.nan_to_num(np.array(list(fidelities[initial_state].values())))
else:
    angles_list = list(interior_points.values())[0]
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    fidelities_list = np.nan_to_num(np.array([fidelities[initial_state][angle] for angle in angles_list]))

In [ ]:
infidelities = 1-fidelities_list
fidelity_ratios = [fidelity_qec/fidelity_noqec for (fidelity_noqec, fidelity_qec) in fidelities_list]
infidelity_ratios = [infidelity_qec/infidelity_noqec for (infidelity_noqec, infidelity_qec) in infidelities]

In [ ]:
print(fidelities_list[0], infidelities[0], infidelity_ratios[0])

In [ ]:
fig = plt.figure(dpi=128)
ax = fig.add_subplot(projection="3d")
ax.set_aspect("equal")

R = 1
u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:30j]
x = R*np.cos(v) * np.sin(u)
y = R*np.sin(v) * np.sin(u)
z = R*np.cos(u)
ax.plot_surface(x, y, z, cmap=plt.cm.bone)

R *= 1.1
xs = R*np.cos(phis + np.pi * 1.2) * np.sin(thetas)
ys = R*np.sin(phis + np.pi * 1.2) * np.sin(thetas)
zs = R*np.cos(thetas)
scatter = ax.scatter(xs, ys, zs, c=infidelity_ratios, cmap=one_centered_cmp, vmin=min(infidelity_ratios), vmax=2-min(infidelity_ratios), s=30)

ax.set_title("Infidelity ratio vs. error angle")

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

plt.colorbar(scatter)

plt.show()

## Plot 2: Fine $\ket{0}$ pole scan

In [ ]:
interior_points, fidelities, effective_threshold = compute_effective_threshold(
    None, **steane_code,
    min_theta=0, max_theta=np.pi/2, n_theta=32,
    min_phi=0, max_phi=np.pi, n_phi=16,
    shots=1E3
)

In [ ]:
initial_state = "0"
plot_all = False

if plot_all:
    angles_list = list(fidelities[initial_state].keys())
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    fidelities_list = np.nan_to_num(np.array(list(fidelities[initial_state].values())))
else:
    angles_list = list(interior_points.values())[0]
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    fidelities_list = np.nan_to_num(np.array([fidelities[initial_state][angle] for angle in angles_list]))

In [ ]:
infidelities = 1-fidelities_list
fidelity_ratios = [fidelity_qec/fidelity_noqec for (fidelity_noqec, fidelity_qec) in fidelities_list]
infidelity_ratios = [infidelity_qec/infidelity_noqec for (infidelity_noqec, infidelity_qec) in infidelities]

In [ ]:
fig, ax = plt.subplots(dpi=128)
ax.set_aspect("equal")

R = 1.1
xs = R*np.cos(phis) * np.sin(thetas)
ys = R*np.sin(phis) * np.sin(thetas)
scatter = ax.scatter(xs, ys, c=infidelity_ratios, cmap=one_centered_cmp, vmin=min(infidelity_ratios), vmax=2-min(infidelity_ratios), s=64)

ax.set_title("Infidelity ratio (QEC/No-QEC) vs. error coordinates")

ax.set_xlabel("X")
ax.set_ylabel("Y")

plt.colorbar(scatter)

plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=128)

scatter = ax.scatter(phis, thetas, c=infidelity_ratios, cmap=one_centered_cmp, vmin=min(infidelity_ratios), vmax=2-min(infidelity_ratios), s=64)

ax.set_title("Infidelity ratio (QEC/No-QEC) vs. error angles")

ax.set_xlabel("$\\phi$")
ax.set_ylabel("$\\theta$")

plt.colorbar(scatter)

plt.show()